# Building your first fully connected network and a CNN 

## Building a simple fully connected network (a Multi-Layer Perceptron)

Let's set up the paths and make a dataset again:

In [1]:
import os,sys
currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

In [2]:
from io_utils.data_handling import simple_np_ds

In [3]:
from utils.engine import Engine

Let's first create a configuration object -we'll use this to set up our training engine

In [4]:
class CONFIG:
    pass
config=CONFIG()
config.batch_size_test = 1024
config.batch_size_train = 128
config.batch_size_val = 2048
config.lr=0.001
config.device = 'gpu'
config.gpu_list=[2,3]
config.num_workers_train=8
config.num_workers_val=8
config.num_workers_test=8
config.dump_path = '../model_state_dumps'


## Defining a simple Convolutional Network

Let's open [simpleCNN](http://localhost:8888/edit/models/simpleCNN.py)

In [5]:
from models.densenet import densenet121
model_CNN=densenet121(num_input_channels=6,num_classes=2)

importing densenet


In [6]:
#import numpy as np
#def rotate_chan(x):
#    return np.transpose(x,(2,0,1))

In [7]:
#dset=simple_np_ds('/fast_scratch/atlas_images/',val_split=0.1,test_split=0.1,transform=rotate_chan)
dset=simple_np_ds('/fast_scratch/atlas_images/',val_split=0.1,test_split=0.1)

labels: [0 0 0 ... 1 1 1]
labels shape: (473521,)


In [8]:
engine=Engine(model_CNN,dset,config)

Requesting GPUs. GPU list : [2, 3]
Main GPU : cuda:2
Using DataParallel on these devices: ['cuda:2', 'cuda:3']
CUDA is available
Creating a directory for run dump: ../model_state_dumps/20191222_061220/


In [9]:
for name, param in model_CNN.named_parameters():
    print("name of a parameter: {}, type: {}, parameter requires a gradient?: {}".
          format(name, type(param),param.requires_grad))

name of a parameter: features.conv0.weight, type: <class 'torch.nn.parameter.Parameter'>, parameter requires a gradient?: True
name of a parameter: features.norm0.weight, type: <class 'torch.nn.parameter.Parameter'>, parameter requires a gradient?: True
name of a parameter: features.norm0.bias, type: <class 'torch.nn.parameter.Parameter'>, parameter requires a gradient?: True
name of a parameter: features.denseblock1.denselayer1.norm1.weight, type: <class 'torch.nn.parameter.Parameter'>, parameter requires a gradient?: True
name of a parameter: features.denseblock1.denselayer1.norm1.bias, type: <class 'torch.nn.parameter.Parameter'>, parameter requires a gradient?: True
name of a parameter: features.denseblock1.denselayer1.conv1.weight, type: <class 'torch.nn.parameter.Parameter'>, parameter requires a gradient?: True
name of a parameter: features.denseblock1.denselayer1.norm2.weight, type: <class 'torch.nn.parameter.Parameter'>, parameter requires a gradient?: True
name of a parameter

In [10]:
%%time
engine.train(epochs=200,report_interval=10,valid_interval=100)

Epoch 0 Starting @ 2019-12-22 06:12:31


RuntimeError: Caught RuntimeError in replica 0 on device 2.
Original Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/torch/nn/parallel/parallel_apply.py", line 60, in _worker
    output = module(*input, **kwargs)
  File "/opt/conda/lib/python3.6/site-packages/torch/nn/modules/module.py", line 545, in __call__
    result = self.forward(*input, **kwargs)
  File "/home/wfedorko/ATLAS_calo/caloml-atlas-upscale/models/densenet.py", line 115, in forward
    features = self.features(x)
  File "/opt/conda/lib/python3.6/site-packages/torch/nn/modules/module.py", line 545, in __call__
    result = self.forward(*input, **kwargs)
  File "/opt/conda/lib/python3.6/site-packages/torch/nn/modules/container.py", line 92, in forward
    input = module(input)
  File "/opt/conda/lib/python3.6/site-packages/torch/nn/modules/module.py", line 545, in __call__
    result = self.forward(*input, **kwargs)
  File "/opt/conda/lib/python3.6/site-packages/torch/nn/modules/container

In [12]:
model_CNN

ResNet(
  (conv1): Conv2d(6, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 